In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import json


/new/benpyenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
models = {
    "TinyLlama": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    "Phi-2": "microsoft/phi-2",
    # "Mistral-7B": "mistralai/Mistral-7B-Instruct-v0.1",
    # "Zephyr-7B": "HuggingFaceH4/zephyr-7b-beta",
    "GPT-Neo-1.3B": "EleutherAI/gpt-neo-1.3B"
}

prompts = {
    "Prompt 1": "Explain the concept of overfitting in machine learning to a high school student.",
    "Prompt 2": "Write a short story involving a robot learning to feel emotions for the first time."
}


In [ ]:
results = {}

for model_name, model_id in models.items():
    print(f"\nLoading model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        load_in_8bit=True if torch.cuda.is_available() else False,
        device_map="auto" if torch.cuda.is_available() else None)

    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

    results[model_name] = {}
    for prompt_name, prompt_text in prompts.items():
        print(f"Generating response from {model_name} for {prompt_name}")
        response = generator(prompt_text, max_new_tokens=200, do_sample=True, temperature=0.7)[0]['generated_text']
        results[model_name][prompt_name] = response


Loading model: TinyLlama


Device set to use cpu


Generating response from TinyLlama for Prompt 1
Generating response from TinyLlama for Prompt 2

Loading model: Phi-2


Loading checkpoint shards: 100%|██████████████████████████████████████| 2/2 [00:05<00:00,  2.60s/it]
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generating response from Phi-2 for Prompt 1


In [5]:
results

{'TinyLlama': {'Prompt 1': 'Explain the concept of overfitting in machine learning to a high school student. Demonstrate how overfitting can occur and provide examples to illustrate how it can impact data analysis.',
  'Prompt 2': 'Write a short story involving a robot learning to feel emotions for the first time.'},
 'Phi-2': {'Prompt 1': "Explain the concept of overfitting in machine learning to a high school student.\n\nSolution:\nOverfitting is a problem that can occur in machine learning when a model is trained too well on the training data, but performs poorly on the testing or unseen data. It's like if you were trying to learn how to ride a bike by only practicing on a small training track, but then tried to ride on a big mountain trail and couldn't keep your balance.\n\nTo understand overfitting, imagine you have a backpack full of different items that you want to organize. You have a limited amount of space in your backpack, so you need to make sure you choose the most importa

In [ ]:
with open("llm_outputs.json", "w") as f:
    json.dump(results, f, indent=4)

In [ ]:
for model_name in results:
    print(f"\n=== {model_name} ===")
    for prompt_name in results[model_name]:
        print(f"\n[{prompt_name}]\n{results[model_name][prompt_name]}\n")